In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
data_path = '/Users/danielsullivan/LinkedIn Learning/Spark SQL/data'

In [4]:
json_df2_path = data_path + "/utilization.json"
df_util = spark.read.format("json").load(json_df2_path)

In [5]:
df_util.show(10)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.77|03/16/2019 17:21:40|       0.22|      115|           58|
|           0.53|03/16/2019 17:26:40|       0.23|      115|           64|
|            0.6|03/16/2019 17:31:40|       0.19|      115|           82|
|           0.46|03/16/2019 17:36:40|       0.32|      115|           60|
|           0.77|03/16/2019 17:41:40|       0.49|      115|           84|
|           0.62|03/16/2019 17:46:40|       0.31|      115|           73|
|           0.71|03/16/2019 17:51:40|       0.54|      115|           67|
|           0.67|03/16/2019 17:56:40|       0.54|      115|           83|
|           0.72|03/16/2019 18:01:40|       0.26|      115|           68|
|           0.62|03/16/2019 18:06:40|       0.52|      115|           60|
+---------------+-------------------+-

In [6]:
df_util.createOrReplaceTempView("utilization")

In [7]:
df_util.count()

500000

In [8]:
df_util.describe().show()

+-------+-------------------+-------------------+-------------------+-----------------+------------------+
|summary|    cpu_utilization|     event_datetime|        free_memory|        server_id|     session_count|
+-------+-------------------+-------------------+-------------------+-----------------+------------------+
|  count|             500000|             500000|             500000|           500000|            500000|
|   mean| 0.6205177399999874|               null|  0.379128099999989|            124.5|          69.59616|
| stddev|0.15875173872912948|               null|0.15830931278376226|14.43088412055312|14.850676696352853|
|    min|               0.22|03/05/2019 08:06:14|                0.0|              100|                32|
|    max|                1.0|04/09/2019 01:22:46|               0.78|              149|               105|
+-------+-------------------+-------------------+-------------------+-----------------+------------------+



In [9]:
df_util.describe('cpu_utilization','free_memory').show()

+-------+-------------------+-------------------+
|summary|    cpu_utilization|        free_memory|
+-------+-------------------+-------------------+
|  count|             500000|             500000|
|   mean| 0.6205177399999874|  0.379128099999989|
| stddev|0.15875173872912948|0.15830931278376226|
|    min|               0.22|                0.0|
|    max|                1.0|               0.78|
+-------+-------------------+-------------------+



In [10]:
df_util.stat.corr('cpu_utilization','free_memory')

-0.47047715730807027

In [11]:
df_util.stat.corr('session_count','free_memory')

-0.5008320848876566

In [12]:
df_util.stat.freqItems(('server_id','session_count')).show()

+--------------------+-----------------------+
| server_id_freqItems|session_count_freqItems|
+--------------------+-----------------------+
|[146, 137, 101, 1...|   [92, 101, 83, 104...|
+--------------------+-----------------------+



In [13]:
df_util_sample = df_util.sample(fraction=0.05, withReplacement=False)
df_util_sample.count()

24879

In [14]:
spark.sql('SELECT min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) FROM utilization').show()

+--------------------+--------------------+----------------------------+
|min(cpu_utilization)|max(cpu_utilization)|stddev_samp(cpu_utilization)|
+--------------------+--------------------+----------------------------+
|                0.22|                 1.0|         0.15875173872912948|
+--------------------+--------------------+----------------------------+



In [15]:
spark.sql('SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
           FROM utilization \
           GROUP BY server_id').show()

+---------+--------------------+--------------------+----------------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev_samp(cpu_utilization)|
+---------+--------------------+--------------------+----------------------------+
|      112|                0.52|                0.92|         0.11528867845082576|
|      113|                0.58|                0.98|         0.11544345150353694|
|      130|                0.35|                0.75|         0.11568834774246008|
|      126|                0.48|                0.88|         0.11542612970702051|
|      149|                0.54|                0.94|         0.11543517500295467|
|      110|                0.35|                0.75|         0.11533251724450215|
|      136|                0.41|                 0.8|         0.11597405743182258|
|      144|                0.47|                0.87|         0.11478654960489501|
|      119|                0.22|                0.62|         0.11516031929842008|
|   

In [16]:
spark.sql('SELECT server_id, FLOOR(cpu_utilization*100/10) bucket FROM utilization').show()

+---------+------+
|server_id|bucket|
+---------+------+
|      115|     7|
|      115|     5|
|      115|     6|
|      115|     4|
|      115|     7|
|      115|     6|
|      115|     7|
|      115|     6|
|      115|     7|
|      115|     6|
|      115|     5|
|      115|     5|
|      115|     5|
|      115|     8|
|      115|     6|
|      115|     8|
|      115|     6|
|      115|     6|
|      115|     6|
|      115|     5|
+---------+------+
only showing top 20 rows

